
## MUJER

En esta notebook analizamos un poco los resultados sobre la categoría WOMEN. Ver [Resultados](Results_full.ipynb)


In [7]:

%load_ext autoreload
%autoreload 2
import pandas as pd
from glob import glob
from rioplatense_hs.mixtral import post_process_output
from rioplatense_hs.preprocessing import labels as all_labels
from rioplatense_hs.preprocessing import text_to_label, labels
from sklearn.metrics import classification_report



df_regionalism = pd.read_csv("../data/regionalisms/check_regionalisms_annotated.csv", index_col=0)

df_regionalism = df_regionalism.rename(columns={"CHECK": "is_regional"})

indices = df_regionalism[df_regionalism["is_regional"] > 0].index

len(indices)


pd.options.display.max_colwidth = 1000
#df_train.loc[df_train["LGBTI"] > 0, ["title", "text", "text_label"]].sample(20)

labels = ["WOMEN", "LGBTI", "RACISM", "CLASS"]


files = {
    #"one-shot": sorted(glob("../data/test*pred*1shot.csv")),
    "gpt-3.5": sorted(glob("../data/runs/test_*_chatgpt.csv")),
    "beto": sorted(glob("../data/runs/test*beto.csv")),
    "aya": sorted(glob("../data/runs/test*aya.csv")),
    "mixtral": sorted(glob("../data/runs/test*mixtral.csv")),
}

pred_labels = [f"PRED_{label}" for label in labels]
score_labels = [f"SCORE_{label}" for label in labels]

dfs = {k: pd.concat([pd.read_csv(f, index_col=0) for f in v]) for k, v in files.items()}
dfs["mixtral"]["pred_cot"] = dfs["mixtral"]["pred_cot"].apply(post_process_output)
{k: v.shape for k, v in dfs.items()}

dfs["beto"][score_labels] = dfs["beto"][pred_labels]
dfs["beto"][pred_labels] = (dfs["beto"][score_labels] > 0.5).astype(int)

#assert len({len(value) for value in files.values()}) == 1



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# Merge dfs['few-shot'] and dfs['beto']

columns_to_drop = [
    "title", "text", "context_tweet",
    # Remove gold labels
    "HATEFUL", "CALLS", "WOMEN", "LGBTI", "RACISM", "CLASS", "POLITICS", "DISABLED", "CRIMINAL", "APPEARANCE",
    # Remove predictions of dismissed labels
    "PRED_CALLS", "PRED_POLITICS", "PRED_DISABLED", "PRED_CRIMINAL", "PRED_APPEARANCE",
]

base_df = dfs["gpt-3.5"].copy()

base_df.drop([
    'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL',
    'PRED_POLITICS',
    'PRED_DISABLED', 'PRED_APPEARANCE', 'PRED_CRIMINAL', 'PRED_HATEFUL'
], axis=1, inplace=True)

print(base_df.columns)
df_join = base_df.join(dfs["beto"].drop(columns_to_drop, axis=1), rsuffix="_beto", lsuffix="_gpt")

mixtral = dfs["mixtral"].copy()

mixtral.drop([c for c in mixtral.columns if c in columns_to_drop], axis=1, inplace=True)
mixtral.rename(columns={f"PRED_{label}": f"PRED_{label}_mixtral" for label in all_labels}, inplace=True)
#mixtral.rename(columns={"pred_cot": "pred_cot_mixtral", "prompt": "prompt_mixtral"}, inplace=True)

df_join = df_join.join(mixtral, rsuffix="_mixtral", lsuffix="_gpt")

cols = [c for c in df_join.columns if c.isupper() and "SCORE" not in c]
#
df_join[cols] = df_join[cols].astype(bool)

sorted(df_join.columns)

Index(['title', 'text', 'context_tweet', 'HATEFUL', 'CALLS', 'WOMEN', 'LGBTI',
       'RACISM', 'CLASS', 'prompt', 'pred_cot', 'PRED_WOMEN', 'PRED_LGBTI',
       'PRED_RACISM', 'PRED_CLASS'],
      dtype='object')


['CALLS',
 'CLASS',
 'HATEFUL',
 'LGBTI',
 'PRED_CLASS_beto',
 'PRED_CLASS_gpt',
 'PRED_CLASS_mixtral',
 'PRED_HATEFUL',
 'PRED_LGBTI_beto',
 'PRED_LGBTI_gpt',
 'PRED_LGBTI_mixtral',
 'PRED_RACISM_beto',
 'PRED_RACISM_gpt',
 'PRED_RACISM_mixtral',
 'PRED_WOMEN_beto',
 'PRED_WOMEN_gpt',
 'PRED_WOMEN_mixtral',
 'RACISM',
 'SCORE_CLASS',
 'SCORE_LGBTI',
 'SCORE_RACISM',
 'SCORE_WOMEN',
 'WOMEN',
 'context_tweet',
 'error',
 'pred_cot_gpt',
 'pred_cot_mixtral',
 'prompt_gpt',
 'prompt_mixtral',
 'text',
 'title']

Ejemplos que falla BETO y acierta ChatGPT 

In [9]:
label = "WOMEN"

beto_false_negatives_detected_by_chatgpt = df_join.loc[
    df_join[label] & df_join[f"PRED_{label}_gpt"] & ~df_join[f"PRED_{label}_beto"]
]

columns_to_show = ["context_tweet", "text", "pred_cot_gpt", "pred_cot_mixtral", f"SCORE_{label}"]

beto_false_negatives_detected_by_chatgpt[columns_to_show].sample(5)

,context_tweet,text,pred_cot_gpt,pred_cot_mixtral,SCORE_WOMEN
id,,,,,
369251,La crítica de Juana Viale a la foto de Alberto Fernández y Hugo Moyano https://t.co/jYpE7KEszY https://t.co/r8crp9DKKP,"@usuario Son negros #Juana y no prestes mucho atención Juana Azurduy,,concubina del presidente no la observe mucho porque de dara un #ACV. Impresentable,amerindia,bachillerato arañando . Que Bonita imagen brindamos al mundo. Primera,dama una borruquita rodillas gastadas.","El texto contiene un discurso de odio hacia personas de raza negra (""son negros""), hacia Juana Azurduy (""concubina del presidente, amerindia""), aludiendo además a su aspecto físico (""borruquita, rodillas gastadas""). También contiene insultos hacia la presidenta de otro país (""Juana""), y al nivel educativo (""bachillerato arañando""), lo cual puede considerarse clasista. La respuesta final es ""racismo, mujer, clase"".","El texto contiene varios insultos y descalificaciones hacia Juana Viale, Hugo Moyano y el presidente Alberto Fernández. En particular, se refiere a Juana Viale como ""negra"" y ""amerindia"", lo que implica un desprecio por su ascendencia indígena. También se refiere a ella como ""borruquita"" y ""bachillerato arañando"", lo que implica un desprecio por su educación y origen social. La respuesta final es ""racismo, clase"".",0.104071
364961,"La otra batalla mundial, el otro enemigo invisible: “La violencia contra las mujeres es pandemia hace siglos” | Por Thelma Fardin https://t.co/mHSJ3PFKEh",@usuario Tranquila si te morís por coronavirus ya nadie podrá violentarte,"El texto hace referencia a la violencia de género que sufren las mujeres y de manera extremadamente insensible y cruel, sugiere que la muerte por coronavirus sería una solución a la violencia que sufren. Aunque no se dirige a una mujer específica, la referencia al contexto de la violencia que padecen las mujeres hace que sea un comentario misógino y despectivo. Por lo tanto, la respuesta final es ""mujer"".","El texto celebra la muerte de una mujer por el coronavirus, lo que implica un desprecio hacia las mujeres. La respuesta final es ""mujer"".",0.003234
327571,Coronavirus y encierro forzoso: advierten que puede aumentar la violencia contra mujeres y niños https://t.co/Sg0V0qNAhp,"@usuario Las feministas están RE Ardidas porque esta situación nos retrotrae a "" la familia unida"" , mamá papá hijos y una mesa para compartir😄\nAh! Si hablas de violencia hacia la mujer, no te olvides de remarcar que la violencia en los niños es ejercida en mayor medida por las madres.","El texto ataca y descalifica a las feministas, llamándolas ""RE Ardidas"", lo cual tiene una connotación negativa y despectiva. Además, minimiza y desvía la atención del problema real de la violencia contra las mujeres al sugerir que la violencia en los niños es ejercida mayormente por las madres, lo cual es incorrecto y desinformado. Por lo tanto, el texto contiene discurso de odio hacia las mujeres. La respuesta final es ""mujer"".","El texto muestra un desprecio hacia el movimiento feminista y hacia las madres. La respuesta final es ""mujer"".",0.012023
341372,Coronavirus: Florencia Kirchner compartió una curiosa reflexión sobre la cuarentena https://t.co/0nI5GKR1NL https://t.co/8KSATPqhFK,@usuario Devolve la plata bicho canasto! Que si mamita no fuera quien es estarías pagando badoo premium a ver si algún ciego te toca y laburando vendiendo productos a domicilio,"El texto hace referencia a Florencia Kirchner de manera insultante por su aspecto físico (""bicho canasto"") y la acusa de recibir dinero ilegítimamente (""devolvé la plata""). Además, hace alusión a su madre (""si mamita no fuera quien es""), lo cual puede considerarse despectivo. En este caso, se muestra un discurso de odio hacia una mujer y hacia la familia de la misma, aunque de manera indirecta. Por lo tanto, la respuesta final es ""mujer"".","El texto critica a Florencia Kirchner por su origen familiar y su situación económica. Además, la despre

In [10]:

not_detected_by_any = df_join.loc[
    df_join[label] & ~df_join[f"PRED_{label}_gpt"] & ~df_join[f"PRED_{label}_beto"]
]

not_detected_by_any[columns_to_show]

,context_tweet,text,pred_cot_gpt,pred_cot_mixtral,SCORE_WOMEN
id,,,,,
388216,El inédito cruce entre la China Suárez y Nancy Pazos por el jubilado de Quilmes https://t.co/rPv5ybYwxG https://t.co/Zi31jyVYVI,@usuario La bella y la bestia ....,"El texto utiliza un calificativo positivo para referirse a China Suárez (""la bella"") y un calificativo negativo para Nancy Pazos (""la bestia""), pero no hay ninguna referencia a una característica protegida. La respuesta final es ""nada"".","El texto compara a Nancy Pazos con una bestia, lo que implica una descalificación de su aspecto físico. La respuesta final es 'mujer'.",0.005037
404354,Británica llegó a Perú por 10 días y se quedó por amor: vive sin agua potable y entre insectos #AMiNoMeImportaElDinero #TengoLoQueYoMásQuieroAMiLado #ElAmorLeGanaATodo? #ConocéLaHistoria!!! https://t.co/1AvKrD7OJb,@usuario La gaver del peruano. Mamá.,"El texto contiene un comentario despectivo hacia los peruanos, utilizando un término vulgar y discriminatorio. La respuesta final es ""racismo"".","El texto se refiere a los peruanos como ""gaver"", un término despectivo. La respuesta final es ""racismo"".",0.007194
351669,El exabrupto de una Diputada bonaerense sobre las personas que violan la cuarentena: “Son hijos de puta y pelotudos pero tienen derecho a un respirador” | Por Matías Russo Coroman https://t.co/Qs2C6KSprg,"@usuario Impresentable, anda a una peluquería","El texto contiene un insulto y una sugerencia despectiva (""anda a una peluquería"") pero no menciona nada relacionado a las características protegidas. La respuesta final es ""nada"".","El texto critica a la diputada por ir a una peluquería, pero no hace ninguna alusión a una característica protegida. La respuesta final es 'nada'.",0.002395
406499,La aberrante arenga machista que dio el preparador físico de Güemes de Santiago del Estero https://t.co/xmV5xrlu9x,@usuario Unos mogolicos los que arengan son re delirantes te crees que vas a ganar por decir hoy me garcho a la mas linda hoy vine caminando o mis viejos se rompieron el culo...\nO sos bueno o sos un mediocre.\nPunto.,"El texto contiene términos despectivos hacia las personas a las que se refiere (""mogolicos"" y ""delirantes"") pero no incluye discursos de odio hacia ninguna de las características protegidas. La respuesta final es ""nada"".","El texto contiene un discurso machista, alentando a los jugadores a tener relaciones sexuales con las mujeres más atractivas y despreciando a los que no lo son. La respuesta final es ""mujer"".",0.005850
368983,"Romina Manguel apuntó contra Susana Giménez: ""Mucha gente la quiere y a mí me incomoda"" https://t.co/kyczDYLGfZ",@usuario Romina buscate un novio. No seas rencorosa.,"El texto contiene una sugerencia sexista hacia Romina Manguel al pedirle que se consiga un novio, aunque no se refiere a ninguna de las características protegidas mencionadas. La respuesta final es ""nada"".",El texto no hace ninguna alusión a una característica protegida. La respuesta final es 'nada'.,0.356839
330557,"😂[QUIEN PUDIERA]😂 Desopilante respuesta de More Rial cuando le preguntaron de qué trabaja: ""Me la rasco..."" &gt;&gt;&gt; En medio de una transmisión de Instagram reveló cuáles son sus hábitos laborales y sorprendió a todos sus seguidores. ¡Mirá!\n\nhttps://t.co/0KWlc9ecBm","@usuario Más que desopilante, es una vergüenza que le aplaudan la vagancia a esta bolsa de papas. Iba a decir "" Se rasca los huevos como Rial""...pero no...osea....no tiene....no hay. ..solo dos bolsitas vacías....sentiende?","El texto contiene una clara muestra de desprecio hacia More Rial, aludiendo a su aspecto físico y comparándola con una ""bolsa de papas"". Además, utiliza un lenguaje despectivo para referirse a su supuesta falta de trabajo y la compara con ""dos bolsitas vacías"". A pesar de no ser directamente una característica protegida, el contenido del mensaje muestra un claro discurso de odio basado en la discriminación y el menosprecio. Por lo tanto, la respuesta fin

In [14]:
# Show those which have one of these
# - Labeled as LGBTI
# - Detected by chatgpt as LGBTI
# - Detected by BETO as LGBTI
# - Detected by Mixtral

df_join["regional"] = df_join.index.isin(indices)

columns = [
    "context_tweet", "text", label,
    f"PRED_{label}_gpt", f"PRED_{label}_beto", f"PRED_{label}_mixtral",
    "pred_cot_gpt", "pred_cot_mixtral", "regional",
]
df_join.loc[
    df_join[label] | df_join[f"PRED_{label}_gpt"] | df_join[f"PRED_{label}_beto"] | df_join[f"PRED_{label}_mixtral"],
    columns
].to_csv(f"../data/tables/{label}_analysis.csv")